In [2]:
!pip install python-Levenshtein

In [3]:
# モジュールのインポート
import json
import urllib.request as req
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
from time import sleep
import Levenshtein
import urllib

In [4]:
base_url = 'https://ja.wikisource.org/'
work_list = [
    '古今和歌集',
    '新古今和歌集',
]

In [5]:
text_list = []
for work in work_list:
    url = os.path.join(
        base_url,
        'wiki/'+urllib.parse.quote(work)
    )
    res = req.urlopen(url)
    sleep(1)
    soup = BeautifulSoup(res, 'html.parser')
    # リンクのみを回収
    link_list = []
    for a in soup.select('a[href]'):
        href = a.attrs['href']
        title = a.string
        link_list.append((title, href))
    # 和歌があるリンクのみを回収
    waka_link_list = []
    for title, href in link_list:
        if isinstance(title, str):
            if ('巻' in title) or (title == '墨滅歌'):
                waka_link_list.append(os.path.join(base_url,href[1:]))
    for url in waka_link_list:
        res = req.urlopen(url)
        soup = BeautifulSoup(res, 'html.parser')
        paragraphs = soup.select('p')
        cnt = -1
        for i,p in enumerate(paragraphs):
            text_list.append(p.get_text().replace('\n',''))
        sleep(1)

In [6]:
data = {
    'kanji':[],
    'hiragana_sep':[]
}
for i,text in enumerate(text_list):
    if '－' in text:
        data['hiragana_sep'].append(text)
        data['kanji'].append(text_list[i-1])

In [7]:
df = pd.DataFrame(data)

In [8]:
def replace_kunojiten(text):
    return text.replace('ほの〳〵','ほのぼの').replace('たえ〳〵','たえだえ').replace('つく〳〵','つくづく').replace('なか〳〵','中々').replace('かへす〳〵','返す返す').replace('まにまに〳〵','').replace('むら〳〵','むらむら').replace('中〳〵','中々').replace('いろ〳〵','色々').replace('よな〳〵','夜な夜な').replace('かつ〳〵','かつかつ').replace('かす〳〵','数々').replace('夜な〳〵','夜な夜な').replace('きり〳〵','きりぎり').replace('しは〳〵','しわしわ').replace('すゑ〳〵','すえずえ').replace('かき〳〵','がきかき').replace('はる〳〵','はるばる').replace('なく〳〵','なくなく').replace('よる〳〵','よるよる').replace('きぬ〳〵','きぬぎぬ').replace('よゐ〳〵','よひよひ').replace('かた〳〵','かたかた').replace('かれ〳〵','かれがれ').replace('さま〳〵','さまざま').replace('なれ〳〵','なれなれ').replace('あさな〳〵','あさなあさな').replace('うら〳〵','うらうら').replace('つれ〳〵','徒然').replace('いまや〳〵','いまやいまや').replace('もろ〳〵','もろもろ').replace('はし〳〵','はしはし')
df['kanji'] = df['kanji'].apply(lambda x:replace_kunojiten(x))
def replace_ichinojiten(text):
    while text.find('ゝ') > 0:
        index = text.find('ゝ')
        replace_charcter = text[index-1]
        text = text[:index] + replace_charcter + text[index+1:]
    return text
df['kanji'] = df['kanji'].apply(lambda x:replace_ichinojiten(x))

In [9]:
def seperate_kanji_waka(kanji_waka,seperated_hiragana_waka):
    hiragana_phrase_list = seperated_hiragana_waka.split('－')
    seperated_kanji_phrase_list = []
    i = 0
    for hiragana_phrase in hiragana_phrase_list:
        levenshtein_list = []
        for j in range(len(kanji_waka)):
            levenshtein_list.append(Levenshtein.distance(kanji_waka[i:j],hiragana_phrase))
            if len(levenshtein_list)>=2 and levenshtein_list[-1] > levenshtein_list[-2]:
                break
        if len(seperated_kanji_phrase_list) == 4:
            seperated_kanji_phrase_list.append(kanji_waka)
        else:
            seperated_kanji_phrase_list.append(kanji_waka[0:j-1])
        kanji_waka = kanji_waka[j-1:]
    return ' '.join(seperated_kanji_phrase_list)

In [10]:
df['kanji'] = df.apply(lambda x: seperate_kanji_waka(x['kanji'], x['hiragana_sep']),axis=1)

In [11]:
df['kaminoku'] = df['kanji'].apply(lambda x: ''.join(x.split(' ')[:3]))
df['shimonoku'] = df['kanji'].apply(lambda x: ''.join(x.split(' ')[3:]))

In [12]:
dataset_list = []
for i,row in enumerate(df.itertuples()):
    dataset_list.append({
        'index': i,
        'instruction': 'input は短歌の上の句です。下の句を詠んでください。',
        'input': row.kaminoku,
        'output': row.shimonoku,
        'category': 'open_qa'
    })

dataset_list.append({
    'index' : dataset_list[-1]['index'] + 1,
    'instruction': 'input は短歌の上の句です。下の句を詠んでください。',
    'input': '父の日にこれ手紙だよ手渡され',
    'output': '読んだら息子の欲しいものリスト',
    'category': 'open_qa'
})
dataset_list.append({
    'index' : dataset_list[-1]['index'] + 1,
    'instruction': 'input は短歌の上の句です。下の句を詠んでください。',
    'input': '事務事故の再発防止禁じ手は',
    'output': '気をつけますと ダブルチェック',
    'category': 'open_qa'
})
dataset_list.append({
    'index' : dataset_list[-1]['index'] + 1,
    'instruction': 'input は短歌の上の句です。下の句を詠んでください。',
    'input': '寝かしつけ全く寝ない子供たち',
    'output': 'その無駄時間俺寝たかった',
    'category': 'open_qa'
})
dataset_list.append({
    'index' : dataset_list[-1]['index'] + 1,
    'instruction': 'input は短歌の上の句です。下の句を詠んでください。',
    'input': 'データあるうまい言葉に騙される',
    'output': '実態いつもガベージコレクション',
    'category': 'open_qa'
})

In [13]:
with open('dataset.json','w') as f:
    f.write(json.dumps(dataset_list, ensure_ascii=False, indent=2))

In [14]:
!head -n15 dataset.json

[
  {
    "index": 0,
    "instruction": "input は短歌の上の句です。下の句を詠んでください。",
    "input": "としのうちに春はきにけりひととせを",
    "output": "こそとやいはむことしとやいはむ",
    "category": "open_qa"
  },
  {
    "index": 1,
    "instruction": "input は短歌の上の句です。下の句を詠んでください。",
    "input": "袖ひちてむすひし水のこほれるを",
    "output": "春立つけふの風やとくらむ",
    "category": "open_qa"
  },


In [15]:
!tail -n29 dataset.json

  {
    "index": 3089,
    "instruction": "input は短歌の上の句です。下の句を詠んでください。",
    "input": "父の日にこれ手紙だよ手渡され",
    "output": "読んだら息子の欲しいものリスト",
    "category": "open_qa"
  },
  {
    "index": 3090,
    "instruction": "input は短歌の上の句です。下の句を詠んでください。",
    "input": "事務事故の再発防止禁じ手は",
    "output": "気をつけますと ダブルチェック",
    "category": "open_qa"
  },
  {
    "index": 3091,
    "instruction": "input は短歌の上の句です。下の句を詠んでください。",
    "input": "寝かしつけ全く寝ない子供たち",
    "output": "その無駄時間俺寝たかった",
    "category": "open_qa"
  },
  {
    "index": 3092,
    "instruction": "input は短歌の上の句です。下の句を詠んでください。",
    "input": "データあるうまい言葉に騙される",
    "output": "実態いつもガベージコレクション",
    "category": "open_qa"
  }
]